In [ ]:
import os
import re
import time
import json
os.environ['GROQ_API_KEY'] = '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' # INSERIR CHAVE DE API GROQ PARA LLAMA GERADA GRATUITAMENTE EM https://console.groq.com/keys
from groq import Groq
client = Groq()

conversation_history = []
filename = ''
div = '-'*500

# ENVIA PROMPT E RETORNA RESPOSTA
def generate_content(prompt, history):
    history.append({
        "role": "user",
        "content": prompt
    })
    response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=history
    )
    rcontent = response.choices[0].message.content
    history.append({
        "role": "assistant",
        "content": rcontent
    })
    return rcontent

# ENVIA PROMPT E SALVA CHAT EM TEXTO EM ARQUIVO
def prompt(history, filename):
    print(div)
    prompt = input("Enter your prompt: ")
    response = generate_content(prompt, history)
    print(response)
    with open(filename, "a", encoding='utf-8') as f:
        f.write(div + "\n")
        f.write("User: " + prompt + "\n")
        f.write("AI: " + response + "\n")
    
# RETORNA O DICIONARIO DE CONTEXTO BASICO (CONVERSA DO ZERO), PARA SER ATRIBUIDO AO conversation_history
def reset():
    # Initialize the conversation history
    history = [
        {
            "role": "user",
            "content": "Olá, como está você?"
        },
        {
            "role": "assistant",
            "content": "Olá! Eu sou um modelo de linguagem de computador, então não tenho sentimentos nem emoções como os humanos, mas estou funcionando normalmente e pronto para ajudar com suas perguntas e tarefas! Como posso ajudá-lo hoje?"
        },
        {
            "role": "user",
            "content": "Você pode me ajudar com questões práticas de programação em C++? Cada vez que eu submeter a descrição de uma questão, quero que você gere um código solução em C++, pronto para compilar."
        },
        {
            "role": "assistant",
            "content": "Sim, estou aqui para ajudar. Submeta a descrição da questão e eu gerarei o código solução em C++."
        }
    ]

    return history

# LE ARQUIVO .JSON E RETORNA DICIONARIO DE CONTEXTO ARMAZENADO, PRONTO PARA SER ATRIBUIDO A UM DICIONARIO PYTHON
def load_history(path):
    #path .json
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# ESCREVE DICIONARIO DE CONTEXTO EM ARQUIVO .JSON
def write_history(history):
    with open('conversation_history.json', 'w', encoding='utf-8') as f:
        json.dump(history, f, ensure_ascii=False, indent=4)

# SUBMETE TEXTO DE CADA QUESTAO DA PASTA /OBI PARA O LLAMA E EXTRAI E SALVA O CODIGO GERADO
def get_solution(path, filename):
    if not filename.endswith('.txt'):
        return
    conversation_history = reset()
    with open(path+'/'+filename, 'r', encoding='utf-8') as f:
        prompt = f.read()
    filename = filename[:-4]+'.cpp'
    response = generate_content(prompt, conversation_history)
    content = response.split('```')[1] # A RESPOSTA DO LLAMA EM CODIGO VEM SEMPRE ENTRE ```
    first_hash_index = content.find('#') # PROCURA O PRIMEIRO INCLUDE
    last_brace_index = content.rfind('}') # PROCURA A ULTIMA CHAVE (FECHAMENTO DA MAIN)
    if first_hash_index == -1 or last_brace_index == -1:
        print(f"File {filename} does not contain required characters.")
        return
    content = content[first_hash_index:last_brace_index + 1]
    with open(path+'/'+filename, "w", encoding='utf-8') as f:
        f.write(content) # ESCREVE CODIGO EM ARQUIVO .cpp	
            



In [12]:
root = './pratique'
for file in os.listdir(root):
    name = file[:-4]
    if name+'.cpp' in root or any(name in s for s in os.listdir('./sol')):
        continue
    get_solution(root, file)